In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SearchandF").getOrCreate()

In [3]:
spark

In [4]:
path = "../datasets/"

fifa = spark.read.csv(path+'fifa19.csv', inferSchema=True, header=True)

In [5]:
fifa.limit(4).toPandas()

22/04/24 22:00:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,75,96,33,28,26,6,11,15,14,8
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,85,95,28,31,23,7,11,15,14,11
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,81,94,27,24,33,9,9,15,15,11
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,40,68,15,21,13,90,85,87,88,94


In [6]:
fifa.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: integer (nullable = true)
 |-- Age: string (nullable = true)
 |-- Photo: integer (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: string (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: integer (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: string (nullable = true)
 |-- Preferred Foot: integer (nullable = true)
 |-- International Reputation: string (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: integer (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: string (nullable = true)
 |-- Joined: integer (nullable = true)
 |-- Loaned From: string (nullable = true)
 |-- Contract Valid 

In [7]:
from pyspark.sql.functions import *

In [8]:
fifa.select(['Nationality','Name', 'Age','Photo']).show(5,False)

+----------------------------------------------+------+-----------------+-----+
|Nationality                                   |Name  |Age              |Photo|
+----------------------------------------------+------+-----------------+-----+
|https://cdn.sofifa.org/players/4/19/158023.png|158023|L. Messi         |31   |
|https://cdn.sofifa.org/players/4/19/20801.png |20801 |Cristiano Ronaldo|33   |
|https://cdn.sofifa.org/players/4/19/190871.png|190871|Neymar Jr        |26   |
|https://cdn.sofifa.org/players/4/19/193080.png|193080|De Gea           |27   |
|https://cdn.sofifa.org/players/4/19/192985.png|192985|K. De Bruyne     |27   |
+----------------------------------------------+------+-----------------+-----+
only showing top 5 rows



In [9]:
fifa.select(['Name', 'Age']).orderBy(fifa['Age']).show(5)

+------+-------------+
|  Name|          Age|
+------+-------------+
|228006|     A. Abang|
|243896|A. Abdellaoui|
|198076| A. Abdennour|
|138698|      A. Abdi|
|245438|A. Abdu Jaber|
+------+-------------+
only showing top 5 rows



In [10]:
fifa.select(['Name', 'Age']).orderBy(fifa['Age'].desc()).show(5)

+------+--------------+
|  Name|           Age|
+------+--------------+
|220433| Óscar Whalley|
|243558|Óscar Valentín|
|208621|   Óscar Plano|
|229865|  Óscar Pinchi|
|246284|     Óscar Gil|
+------+--------------+
only showing top 5 rows



In [11]:
fifa.select(['Name', 'Club']).where(fifa.Club.like("%Barcelona%")).show(5)

+----+----+
|Name|Club|
+----+----+
+----+----+



In [12]:
fifa.select("Photo", fifa.Photo.substr(-4,4).alias("extension")).show(5, False)

+-----+---------+
|Photo|extension|
+-----+---------+
|31   |31       |
|33   |33       |
|26   |26       |
|27   |27       |
|27   |27       |
+-----+---------+
only showing top 5 rows



In [13]:
fifa[fifa.Club.isin("FC Barcelona", "Juventus")].limit(4).toPandas()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause


In [14]:
fifa.select("Name","Club").where(fifa.Name.startswith("Di")).where(fifa.Name.endswith("o")).show(5)

+----+----+
|Name|Club|
+----+----+
+----+----+



In [15]:
fifa.count()

18207

In [16]:
df = fifa.limit(100)
df.count()

100

In [17]:
colList = fifa.columns[0:5]
df3 = fifa.select(colList)

In [18]:
df3.show()

+---+------+-----------------+-----+--------------------+
| ID|  Name|              Age|Photo|         Nationality|
+---+------+-----------------+-----+--------------------+
|  0|158023|         L. Messi|   31|https://cdn.sofif...|
|  1| 20801|Cristiano Ronaldo|   33|https://cdn.sofif...|
|  2|190871|        Neymar Jr|   26|https://cdn.sofif...|
|  3|193080|           De Gea|   27|https://cdn.sofif...|
|  4|192985|     K. De Bruyne|   27|https://cdn.sofif...|
|  5|183277|        E. Hazard|   27|https://cdn.sofif...|
|  6|177003|        L. Modrić|   32|https://cdn.sofif...|
|  7|176580|        L. Suárez|   31|https://cdn.sofif...|
|  8|155862|     Sergio Ramos|   32|https://cdn.sofif...|
|  9|200389|         J. Oblak|   25|https://cdn.sofif...|
| 10|188545|   R. Lewandowski|   29|https://cdn.sofif...|
| 11|182521|         T. Kroos|   28|https://cdn.sofif...|
| 12|182493|         D. Godín|   32|https://cdn.sofif...|
| 13|168542|      David Silva|   32|https://cdn.sofif...|
| 14|215914|  

In [19]:
len(df3.columns)

5

In [20]:
df = spark.createDataFrame([([1,2,3],),([4,5],)],['x'])
df.show()

+---------+
|        x|
+---------+
|[1, 2, 3]|
|   [4, 5]|
+---------+



In [21]:
df.select(slice(df.x,2,2).alias('Some name')).show()

+---------+
|Some name|
+---------+
|   [2, 3]|
|      [5]|
+---------+



In [32]:
fifa.filter("Overall>50").limit(5).toPandas()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause


In [27]:
fifa.filter("Overall>=50").limit(5).select(['Name','Age']).toPandas()

,Name,Age


In [24]:
result = fifa.filter("Overall>50").select(['Nationality','Name','Age']).orderBy(fifa['Overall'].desc()).collect()

In [25]:
type(result[0])

IndexError: list index out of range

In [ ]:
print("Worst Player Over 50: ", result[-1][1])

Bast Player Over 50:  C. Addai
